In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

In [2]:
df = pd.read_csv('airbnb_bologna_0310_2016-02-07.csv')


In [3]:
df.drop(['bathrooms', 'minstay', 'room_id', 'survey_id', 'host_id', 'borough', 'name', 'last_modified', 'latitude', 'longitude', 'location', 'city', 'neighborhood', 'country'], axis=1, inplace=True)
df.head()

room_type  reviews  overall_satisfaction  accommodates  bedrooms  \
0     Private room       17                   5.0           2.0       1.0   
1  Entire home/apt       92                   4.5           5.0       0.0   
2  Entire home/apt       35                   5.0           5.0       3.0   
3  Entire home/apt       35                   5.0           4.0       2.0   
4     Private room        3                   5.0           2.0       1.0   

   price  
0   43.0  
1   79.0  
2  138.0  
3   75.0  
4   43.0

In [4]:
df.isnull().sum()

room_type                 0
reviews                   0
overall_satisfaction    122
accommodates             26
bedrooms                  1
price                     0
dtype: int64

In [5]:
df['overall_satisfaction'].fillna(df['overall_satisfaction'].mode()[0], inplace=True)
df['accommodates'].fillna(df['accommodates'].mode()[0], inplace=True)
df['bedrooms'].fillna(df['bedrooms'].mode()[0], inplace=True)

In [6]:
df.describe()

reviews  overall_satisfaction  accommodates    bedrooms       price
count  736.000000            736.000000    736.000000  736.000000  736.000000
mean    22.902174              4.726902      2.607337    1.084239   65.985054
std     33.926871              0.380846      0.996265    0.527834   41.337878
min      0.000000              2.500000      1.000000    0.000000   11.000000
25%      2.000000              4.500000      2.000000    1.000000   40.000000
50%      9.000000              5.000000      2.000000    1.000000   58.000000
75%     31.000000              5.000000      3.000000    1.000000   80.000000
max    259.000000              5.000000      5.000000    6.000000  749.000000

In [7]:
def change_overall_satisfaction(overall_satisfaction):
    if overall_satisfaction > 4:
        return 'a. Excellent'
    elif overall_satisfaction > 3:
        return 'b. Very good'
    elif overall_satisfaction > 2:
        return 'c. Good'
    elif overall_satisfaction > 1:
        return 'd. Fair'
    else:
        return 'e. Poor'
    
df['overall_satisfaction'] = df['overall_satisfaction'].apply(change_overall_satisfaction)

In [8]:
df = pd.get_dummies(df, columns=['room_type'])

In [9]:
df.head()

reviews overall_satisfaction  accommodates  bedrooms  price  \
0       17         a. Excellent           2.0       1.0   43.0   
1       92         a. Excellent           5.0       0.0   79.0   
2       35         a. Excellent           5.0       3.0  138.0   
3       35         a. Excellent           4.0       2.0   75.0   
4        3         a. Excellent           2.0       1.0   43.0   

   room_type_Entire home/apt  room_type_Private room  room_type_Shared room  
0                          0                       1                      0  
1                          1                       0                      0  
2                          1                       0                      0  
3                          1                       0                      0  
4                          0                       1                      0

In [10]:
X = df.drop('overall_satisfaction', axis=1)  
y = df['overall_satisfaction']

In [11]:
from sklearn.model_selection import train_test_split  
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20)

In [12]:
from sklearn.svm import SVC  
svclassifier = SVC(kernel='linear')  
svclassifier.fit(X_train, y_train)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='linear',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [13]:
y_pred = svclassifier.predict(X_test)  

In [14]:
from sklearn.metrics import classification_report, confusion_matrix  
print(confusion_matrix(y_test,y_pred))  
print(classification_report(y_test,y_pred))

[[129   0   0]
 [ 18   0   0]
 [  1   0   0]]
              precision    recall  f1-score   support

a. Excellent       0.87      1.00      0.93       129
b. Very good       0.00      0.00      0.00        18
     c. Good       0.00      0.00      0.00         1

 avg / total       0.76      0.87      0.81       148



C:\Users\user\Anaconda2\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [15]:
from sklearn import metrics

# Get predictions
#predicted = model.predict(X_test)
#expected = Y_test

# Compare results
print (metrics.accuracy_score(y_test,y_pred))

0.871621621622
